In [1]:
from Agent_Global import *

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Hyperparameters
gamma = 0.9
nb_trials = 100
nb_episodes = 10000
learning_rate = 7e-6
eps = 1e-7
beta_v = 0.05
beta_e = 0.05

# Prepare

In [3]:
rail_generator = sparse_rail_generator(max_num_cities=10)

env = RailEnv(
    width=30, 
    height=30, 
    number_of_agents=1,
    rail_generator=rail_generator,
    line_generator=sparse_line_generator(),
    obs_builder_object=GlobalObsForRailEnv()
    )

# env.stop_penalty = -1
# env.invalid_action_penalty = -1
# env.alpha = 1

In [4]:
# Save Paths
path = "train_LSTMCell_A2C/" + datetime.now().strftime("%m%d-%H:%M:%S") + "_adam_lr7e-6_g0.9_punish"
log_dir = path+'/logs/'
ckpt_dir = path+'/ckpt/'
train_summary_writer = tf.summary.create_file_writer(log_dir)

2023-01-16 10:01:30.779063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
agent_id = 0
agent = LSTMAgent(learning_rate, gamma, beta_v, beta_e,  #loss func
                 env, nb_trials, nb_episodes,path,
                 nb_hidden = 128)

# Train

In [6]:
agent.train()

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 1/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:262: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")


KeyboardInterrupt: 

In [ ]:
agent.model.save(path+'/model.h5')

# Test

In [ ]:
#test_model = keras.models.load_model("train_LSTMCell_A2C/0115-19:25:47_adam/model.h5")
test_model = agent.model

In [ ]:

def run_episode(env, model):
    next_obs, _ = env.reset()
    score = 0
    reward = 0.0
    action_onehot = np.zeros((agent.nb_actions))
    cell_state = [tf.zeros((1,agent.nb_hidden)),tf.zeros((1,agent.nb_hidden))]
        
    for timestep in range(100):
        input = agent.prepare_input(next_obs, agent_id, action_onehot,reward,timestep)
        
        action_probs, critic_value, cell_state = model([input,cell_state[0],cell_state[1]])
        
        action_probs = tf.squeeze(action_probs)
        action = np.random.choice(agent.nb_actions, p=action_probs.numpy())
        action_onehot = np.zeros((agent.nb_actions))
        action_onehot[action] = 1

        next_obs, all_rewards, dones, info = env.step({agent_id: action})

        for agent_handle in env.get_agent_handles():
            score += all_rewards[agent_handle]

        #render_env(env)
        #print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
        #tf.print(action_probs)

        if dones['__all__']:
            print('All done!')
            print(info['state'][0])
            print(env.agents[0].latest_arrival, env._max_episode_steps)
            print(env.agents[agent_id].position, env.agents[agent_id].target)

            if info['state'][0] == TrainState.DONE:
                #render_env(env)
                print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
            return score

    print("Episode didn't finish after 100 timesteps.")
    return score

In [ ]:
print(env.agents[0].earliest_departure)
print(env.agents[0].latest_arrival)

0
72


In [ ]:
out = []
for i in range(100):
    out.append(run_episode(env, test_model))

    
print(out)

All done!
TrainState.MOVING
51 57
(14, 20) (9, 16)
All done!
TrainState.DONE
115 124
None (22, 21)
Timestep 80, action = foward, total score = 0
All done!
TrainState.MOVING
81 90
(16, 25) (10, 22)
All done!
TrainState.MOVING
35 37
(17, 22) (10, 19)
Episode didn't finish after 100 timesteps.
All done!
TrainState.MOVING
55 61
(11, 18) (24, 15)
Episode didn't finish after 100 timesteps.
All done!
TrainState.MOVING
24 27
(7, 17) (23, 21)
All done!
TrainState.MOVING
44 47
(14, 22) (10, 17)
All done!
TrainState.DONE
48 53
None (23, 6)
Timestep 32, action = right, total score = 0
All done!
TrainState.DONE
114 121
None (18, 6)
Timestep 85, action = foward, total score = 0
All done!
TrainState.DONE
79 85
None (18, 7)
Timestep 57, action = foward, total score = 0
Episode didn't finish after 100 timesteps.
All done!
TrainState.STOPPED
30 32
(27, 13) (11, 12)
Episode didn't finish after 100 timesteps.
All done!
TrainState.DONE
105 114
None (11, 23)
Timestep 87, action = foward, total score = 0
All